In [2]:
#IMPORT LIBRARIES
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time
import random
import re
from bs4 import BeautifulSoup
import requests
import lxml
from lxml import html
import os
import pandas as pd

## 1.Data Collection

In [ ]:
#1.1- Reach main URL
base_url = 'https://www.findamasters.com/masters-degrees/msc-degrees/?PG=1'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(base_url)
driver.maximize_window()

#WAIT FOR PUBLICITY
try:
    WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='signupModal']/div/div/div/i"))).click()
except:
    print("No pop up")
time.sleep(10)

#MASTER COURSES URL
masters_url = []
base_url = 'https://www.findamasters.com/'
base_url_page = 'https://www.findamasters.com/masters-degrees/msc-degrees/?PG='
for n in range(1,401):
    
    url = base_url_page + str(n)
    res = requests.get(url)
    time.sleep(random.uniform(5, 10))
    soup = BeautifulSoup(res.text, "lxml")
    
    masters = soup.find_all('a', {'class': 'courseLink text-dark', 'href':True})
    for master in masters:
        masters_url.append(base_url + master['href'])

In [ ]:
#1.1- Checking we collected all the required URL's (6000)
print(len(masters_url))

In [ ]:
#1.1- Saving the collected URL's in a txt file
with open('urls.txt', 'w') as file:
    # Write each URL to a new line
    for url in masters_url:
        file.write(url + '\n')

In [1]:
#1.2- Saving our txt folder as a list 
with open('urls.txt', 'r') as file:
    # Read the file line by line
    urls = [line.strip() for line in file]

In [ ]:
#1.2- Download the HTML of a URL and save it as a file in a folder
def download_and_save(url, i):
    try:
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(url)
        time.sleep(5)
        html = driver.page_source
        folder_name = f'folder_{i // 15}'
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        file_name = f'{i % 15}.html'
        with open(os.path.join(folder_name, file_name), 'w', encoding='utf-8') as f:
            f.write(html)
        driver.quit()
    except Exception as e:
        print(f"An error occurred while downloading {url}: {e}")

In [ ]:
#1.2- Using multiple thread to execute the 'download_and_save' function
import concurrent.futures

with concurrent.futures.ThreadPoolExecutor() as executor:
  # Use the executor to map the function to the URLs
  executor.map(download_and_save, urls, range(len(urls)))

### Due to the large number of pages we should download, we decided to use Python's concurrent.futures module which use multiple threads. Using this method we are able to execute the 'download_and_save' function over multiple URL's concurrently and to decrease the running time.

In [3]:
#1.3- Creating a Data Frame with the required columns
df = pd.DataFrame(columns=["courseName", "universityName", "facultyName", "isItFullTime", "description", "startDate", "fee", "modality", "duration", "city", "country", "administration", "url"])

In [4]:
#1.3- Collect the data from each HTML file and add it to the Data Frame
for i in range(400):
    folder_name = f'folder_{i}'
    for j in range(15):
        file_name = f'{j}.html'
        with open(os.path.join(folder_name, file_name), 'r', encoding='utf-8') as f:
            contents = f.read()
            soup = BeautifulSoup(contents, 'html.parser')
        tree = html.fromstring(str(soup))
        e_courseName = tree.xpath('//*[@id="MainArea"]/div[2]/div/div/div/div/div/h1')
        for element in e_courseName:
            courseName = element.text_content()
        e_universityName = tree.xpath('//*[@id="MainArea"]/div[2]/div/div/div/div/div/h3/span/a[1]')
        for element in e_universityName:
            universityName = element.text_content()
        e_facultyName = tree.xpath('//*[@id="MainArea"]/div[4]/div/div/div[1]/div/div[3]/a[1]')
        for element in e_facultyName:
            facultyName = element.text_content()
        e_isItFullTime = tree.xpath('//*[@id="MainArea"]/div[4]/div/div/div[1]/div/div[1]/div/span[1]/a')
        for element in e_isItFullTime:
            isItFullTime = element.text_content()
        e_description = tree.xpath('//*[@id="Snippet"]')
        for element in e_description:
            description = element.text_content()
        e_startDate = tree.xpath('//*[@id="MainArea"]/div[4]/div/div/div[1]/div/div[1]/div/span[2]')
        for element in e_startDate:
            startDate = element.text_content()
        e_fees = tree.xpath('//*[@id="MainArea"]/div[4]/div/div/div[1]/div/div[7]/div/p/a') 
        for element in e_fees:
            fee = element.text_content()
        e_modality = tree.xpath('//*[@id="MainArea"]/div[4]/div/div/div[1]/div/div[1]/div/span[3]')
        for element in e_modality:
            modality = element.text_content()
        e_duration = tree.xpath('//*[@id="MainArea"]/div[4]/div/div/div[1]/div/div[1]/div/span[4]')
        for element in e_duration:
            duration = element.text_content()
        e_city = tree.xpath('//*[@id="MainArea"]/div[4]/div/div/div[1]/div/div[3]/a[4]')
        for element in e_city:
            city = element.text_content()
        e_country = tree.xpath('//*[@id="MainArea"]/div[4]/div/div/div[1]/div/div[3]/a[5]')
        for element in e_country:
            country = element.text_content()
        e_administraion = tree.xpath('//*[@id="MainArea"]/div[4]/div/div/div[1]/div/div[3]/a[6]')
        for element in e_administraion:
            administraion = element.text_content()
        url = urls[i]
        df = df.append({"courseName": courseName, "universityName": universityName, "facultyName": facultyName, "isItFullTime": isItFullTime, "description": description, "startDate": startDate, "fee": fee, "modality": modality, "duration": duration, "city": city, "country": country, "administraion": administraion, "url":url}, ignore_index=True)

In [5]:
#1.3- Checking we added all the data (6000)
print(df.shape[0])

6000


In [6]:
#1.3- Converting the Data Frame to tsv file
df.to_csv('courses.tsv', sep='\t', index=False)

In [7]:
#1.3- Converting the Data Frame to csv file
df.to_csv('output.csv', index=False)